## Instacart Market Basket Analysis
The objective is to predict which customer will repurchase a product in his current order, based on a anonymized dataset containing 3,000,000 orders.

Each user then has about 4 - 100 orders.

A user's reorder depends on User, the products that he purchases and the history of his purchases 

 

## Extraction and Loading 

Lets take  look at a  few datasets provided

In [1]:
% matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Read in the files 

In [2]:
order_products_train_df = pd.read_csv("../../input/order_products__train.csv")
order_products_prior_df = pd.read_csv("../../input/order_products__prior.csv")
orders_df = pd.read_csv("../../input/orders.csv")
products_df = pd.read_csv("../../input/products.csv")
aisles_df = pd.read_csv("../../input/aisles.csv")
departments_df = pd.read_csv("../../input/departments.csv")

#### Product

In [3]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
products_df.shape

(49688, 4)

#### Aisles

In [5]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [6]:
aisles_df.shape

(134, 2)

#### Department

In [7]:
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [8]:
departments_df.shape

(21, 2)

#### Orders

In [ ]:
orders_df.head()

In [ ]:
(rows, cols) = orders_df.shape

In [ ]:
print(" There are {} unique orders in the training set".format(rows))

In [ ]:
orders_df.groupby('eval_set')['order_id'].count()

#### Training set orders

In [ ]:
order_products_train_df.head()

As seen in the table above, for each unique order_id, we have product, the order in which it was ordered and whether this was a product thatwas re-odered.

In [ ]:
order_products_train_df.shape

There are 13.8 Million observations and 4 attribures.
How many unique orders are there ?

In [ ]:
order_products_train_df.groupby('order_id').count().head()

In [ ]:
print(" Order_id  are not contiguous")
(rows,cols) = order_products_train_df.groupby('order_id').count().shape

print(" There are {} unique orders in the training set".format(rows))